In [29]:
import os

import schematics as scm
import schematics.types as scmt

import pandas as pd
import numpy as np
import tensorflow.keras.callbacks as tfkc
import tensorflow.keras.optimizers as tfko
import flatiron.core.tools as fict

%cd ~/flatiron/python

from flatiron.core.pipeline import *

/home/ubuntu/flatiron/python


In [2]:
class FooConfig(scm.Model):
    bar = scmt.StringType(required=True)

class FooPipeline(PipelineBase):
    @property
    def model_config(self):
        return FooConfig
    
    def build(self):
        pass

def create_fake_asset(root):
    chunks = [
        'data/chunk_f00.npy',
        'data/chunk_f01.npy',
    ]
    os.makedirs(root, exist_ok=True)
    os.makedirs(Path(root, 'data'), exist_ok=True)
    data = pd.DataFrame()
    data['filepath_relative'] = chunks
    data['asset_path'] = root
    csv_path = Path(root, 'info.csv').as_posix()
    data.to_csv(csv_path, index=False)
    shape = (100, 10, 10, 3)
    arr = np.ones(shape, dtype=np.float16)
    for chunk in chunks:
        np.save(Path(root, chunk), arr)
    return data

In [3]:
root = '/tmp/foobar'
d = create_fake_asset(root)
config = dict(
    model=dict(bar='baz'),
    dataset=dict(
        source=root,
        # load_limit=100,
        split_index=3,
    )
)
p = FooPipeline(config) \
    .load() \
    .train_test_split() \
    .unload()

p

Loading Dataset Chunks:   0%|          | 0/2 [00:00<?, ?it/s]

((160, 10, 10, 3), (40, 10, 10, 3))

In [21]:
def get_params():
    params = dict(
        info=dict(
            project='proj0121',
            root='/mnt/storage/projects',
            spec='dset001',
            desc='bg-detection',
            version=4,
        ),
        data=dict(
            limit=15000,
        ),
        model=dict(
            input_shape=(192, 192, 3),
            num_classes=1,
            activation='leaky_relu',
            use_batch_norm=True,
            upsample_mode='deconv',
            dropout=0.0,
            dropout_change_per_layer=0.0,
            dropout_type='spatial',
            use_dropout_on_upsampling=False,
            use_attention=True,
            filters=32,
            num_layers=4,
            output_activation='sigmoid',
        ),
        misc=dict(
            optimizer_class=tfko.SGD,
            batch_size=16,
        ),
        optimizer=dict(
            learning_rate=0.008,
            momentum=0.99,
        ),
        compile_=dict(
            loss='jaccard_loss',
            metrics=[],
        ),
        callback=dict(
            save_best_only=True,
            metric='jaccard_coef',
            mode='auto',
            save_freq='epoch',
            update_freq='batch',
        ),
        fit=dict(
            epochs=35,
            verbose='auto',
            shuffle=True,
            initial_epoch=0,
            use_multiprocessing=True,
        ),
        datagen=dict(
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='constant',
        )
    )

    params['compile_']['optimizer'] = params['misc']['optimizer_class'](**params['optimizer'])
    root = params['info']['root']
    proj = params['info']['project']
    cb = params['callback']
    # cbs, log_dir = fict.get_callbacks(root, proj, cb)
    params['misc']['log_dir'] = 'log_dir'
    params['fit']['callbacks'] = ['callbacks']

    return params

def patch_params(params, x_train, x_test, y_train, y_test):
    assert params['model']['input_shape'] == x_train[0].shape
    params['fit']['steps_per_epoch'] = np.ceil(x_train.shape[0] / params['misc']['batch_size'])
    params['fit']['validation_data'] = (x_test, y_test)
    return params

p = get_params()
del p['info']
del p['data']
del p['model']
del p['misc']
del p['datagen']
p['compile_']
p

{'optimizer': {'learning_rate': 0.008, 'momentum': 0.99},
 'compile_': {'loss': 'jaccard_loss',
  'metrics': [],
  'optimizer': <keras.optimizers.optimizer_experimental.sgd.SGD at 0x7f4309aadf90>},
 'callback': {'save_best_only': True,
  'metric': 'jaccard_coef',
  'mode': 'auto',
  'save_freq': 'epoch',
  'update_freq': 'batch'},
 'fit': {'epochs': 35,
  'verbose': 'auto',
  'shuffle': True,
  'initial_epoch': 0,
  'use_multiprocessing': True,
  'callbacks': ['callbacks']}}